In [55]:
import os

import deepl
import pandas as pd
from dotenv import load_dotenv
from tqdm import tqdm
import tiktoken
from openai import OpenAI
import faiss
import numpy as np


# Enable tqdm for pandas
tqdm.pandas()

In [2]:
df = pd.read_json("hf://datasets/HuggingFaceH4/MATH-500/test.jsonl", lines=True)
df

,problem,solution,answer,subject,level,unique_id
0,"Convert the point $(0,3)$ in rectangular coord...",We have that $r = \sqrt{0^2 + 3^2} = 3.$ Also...,"\left( 3, \frac{\pi}{2} \right)",Precalculus,2,test/precalculus/807.json
1,Define\n\[p = \sum_{k = 1}^\infty \frac{1}{k^2...,We count the number of times $\frac{1}{n^3}$ a...,p - q,Intermediate Algebra,5,test/intermediate_algebra/1994.json
2,"If $f(x) = \frac{3x-2}{x-2}$, what is the valu...",$f(-2)+f(-1)+f(0)=\frac{3(-2)-2}{-2-2}+\frac{3...,\frac{14}{3},Algebra,3,test/algebra/2584.json
3,How many positive whole-number divisors does 1...,First prime factorize $196=2^2\cdot7^2$. The ...,9,Number Theory,3,test/number_theory/572.json
4,The results of a cross-country team's training...,Evelyn covered more distance in less time than...,\text{Evelyn},Algebra,2,test/algebra/1349.json
...,...,...,...,...,...,...
495,What is the domain of the function $f(x) = \fr...,The inner logarithm is only defined if $x - 2 ...,"(2,12) \cup (12,102)",Intermediate Algebra,4,test/intermediate_algebra/1981.json
496,Let $z = 1+i$ and $w = \dfrac{3z+1}{5z+7}$. Fi...,"Plugging in, we have $w = \dfrac{3(1+i)+1}{5(1...",\frac{5}{13},Intermediate Algebra,3,test/intermediate_algebra/1232.json
497,An equiangular octagon has four sides of lengt...,The octagon can be partitioned into five squar...,\frac{7}{2},Geometry,5,test/geometry/561.json
498,A sequence $(a_n)$ is defined as follows:\n\[a...,"First, if $a_3 = a_1,$ then\n\[a_1 = a_3 = a_5...",-1,Intermediate Algebra,5,test/intermediate_algebra/1508.json


In [3]:
df.loc[:, "original_level"] = df.loc[:, "level"]
df.loc[:, "level"] = df.loc[:, "level"].replace({
    1: "Basic Math Concepts",
    2: "Foundational High-School Math",
    3: "High-School Intermediate Level Competition Math Problems",
    4: "High-Level Math Competition Problems; Regional Math Competitions.",
    5: "Advanced Math Competitions; International Olympiad or University Courses Level."
})
df

/var/folders/gg/1kd93k4x47q5_gj92ljtjf000000gq/T/ipykernel_89857/212375986.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['Foundational High-School Math'
 'Advanced Math Competitions; International Olympiad or University Courses Level.'
 'High-School Intermediate Level Competition Math Problems'
 'High-School Intermediate Level Competition Math Problems'
 'Foundational High-School Math' 'Foundational High-School Math'
 'High-School Intermediate Level Competition Math Problems'
 'High-Level Math Competition Problems; Regional Math Competitions.'
 'High-School Intermediate Level Competition Math Problems'
 'Advanced Math Competitions; International Olympiad or University Courses Level.'
 'High-School Intermediate Level Competition Math Problems'
 'Advanced Math Competitions; International Olympiad or University Courses Level.'
 'Advanced Math Competitions; International Olympiad or University Courses Level.'
 '

,problem,solution,answer,subject,level,unique_id,original_level
0,"Convert the point $(0,3)$ in rectangular coord...",We have that $r = \sqrt{0^2 + 3^2} = 3.$ Also...,"\left( 3, \frac{\pi}{2} \right)",Precalculus,Foundational High-School Math,test/precalculus/807.json,2
1,Define\n\[p = \sum_{k = 1}^\infty \frac{1}{k^2...,We count the number of times $\frac{1}{n^3}$ a...,p - q,Intermediate Algebra,Advanced Math Competitions; International Olym...,test/intermediate_algebra/1994.json,5
2,"If $f(x) = \frac{3x-2}{x-2}$, what is the valu...",$f(-2)+f(-1)+f(0)=\frac{3(-2)-2}{-2-2}+\frac{3...,\frac{14}{3},Algebra,High-School Intermediate Level Competition Mat...,test/algebra/2584.json,3
3,How many positive whole-number divisors does 1...,First prime factorize $196=2^2\cdot7^2$. The ...,9,Number Theory,High-School Intermediate Level Competition Mat...,test/number_theory/572.json,3
4,The results of a cross-country team's training...,Evelyn covered more distance in less time than...,\text{Evelyn},Algebra,Foundational High-School Math,test/algebra/1349.json,2
...,...,...,...,...,...,...,...
495,What is the domain of the function $f(x) = \fr...,The inner logarithm is only defined if $x - 2 ...,"(2,12) \cup (12,102)",Intermediate Algebra,High-Level Math Competition Problems; Regional...,test/intermediate_algebra/1981.json,4
496,Let $z = 1+i$ and $w = \dfrac{3z+1}{5z+7}$. Fi...,"Plugging in, we have $w = \dfrac{3(1+i)+1}{5(1...",\frac{5}{13},Intermediate Algebra,High-School Intermediate Level Competition Mat...,test/intermediate_algebra/1232.json,3
497,An equiangular octagon has four sides of lengt...,The octagon can be partitioned into five squar...,\frac{7}{2},Geometry,Advanced Math Competitions; International Olym...,test/geometry/561.json,5
498,A sequence $(a_n)$ is defined as follows:\n\[a...,"First, if $a_3 = a_1,$ then\n\[a_1 = a_3 = a_5...",-1,Intermediate Algebra,Advanced Math Competitions; International Olym...,test/intermediate_algebra/1508.json,5


In [4]:
load_dotenv("./../.env")

True

In [5]:
auth_key = os.environ["DEEPL_AUTH_KEY"]
translator = deepl.Translator(auth_key)

result = translator.translate_text("Hello, world!", target_lang="FR")
print(result.text)

Bonjour à tous !


In [8]:
columns_to_count = [
    'problem',
    'solution',
    'subject',
    'level',
]

# Count characters per cell in specified columns
df_char_count = df[columns_to_count].applymap(len)
for column in columns_to_count:
    df[f'{column}_char_count'] = df_char_count[column]

df

/var/folders/gg/1kd93k4x47q5_gj92ljtjf000000gq/T/ipykernel_89857/984970557.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_char_count = df[columns_to_count].applymap(len)


,problem,solution,answer,subject,level,unique_id,original_level,problem_char_count,solution_char_count,subject_char_count,level_char_count
0,"Convert the point $(0,3)$ in rectangular coord...",We have that $r = \sqrt{0^2 + 3^2} = 3.$ Also...,"\left( 3, \frac{\pi}{2} \right)",Precalculus,Foundational High-School Math,test/precalculus/807.json,2,161,439,11,29
1,Define\n\[p = \sum_{k = 1}^\infty \frac{1}{k^2...,We count the number of times $\frac{1}{n^3}$ a...,p - q,Intermediate Algebra,Advanced Math Competitions; International Olym...,test/intermediate_algebra/1994.json,5,217,773,20,79
2,"If $f(x) = \frac{3x-2}{x-2}$, what is the valu...",$f(-2)+f(-1)+f(0)=\frac{3(-2)-2}{-2-2}+\frac{3...,\frac{14}{3},Algebra,High-School Intermediate Level Competition Mat...,test/algebra/2584.json,3,113,158,7,56
3,How many positive whole-number divisors does 1...,First prime factorize $196=2^2\cdot7^2$. The ...,9,Number Theory,High-School Intermediate Level Competition Mat...,test/number_theory/572.json,3,54,506,13,56
4,The results of a cross-country team's training...,Evelyn covered more distance in less time than...,\text{Evelyn},Algebra,Foundational High-School Math,test/algebra/1349.json,2,731,326,7,29
...,...,...,...,...,...,...,...,...,...,...,...
495,What is the domain of the function $f(x) = \fr...,The inner logarithm is only defined if $x - 2 ...,"(2,12) \cup (12,102)",Intermediate Algebra,High-Level Math Competition Problems; Regional...,test/intermediate_algebra/1981.json,4,166,412,20,65
496,Let $z = 1+i$ and $w = \dfrac{3z+1}{5z+7}$. Fi...,"Plugging in, we have $w = \dfrac{3(1+i)+1}{5(1...",\frac{5}{13},Intermediate Algebra,High-School Intermediate Level Competition Mat...,test/intermediate_algebra/1232.json,3,55,442,20,56
497,An equiangular octagon has four sides of lengt...,The octagon can be partitioned into five squar...,\frac{7}{2},Geometry,Advanced Math Competitions; International Olym...,test/geometry/561.json,5,195,630,8,79
498,A sequence $(a_n)$ is defined as follows:\n\[a...,"First, if $a_3 = a_1,$ then\n\[a_1 = a_3 = a_5...",-1,Intermediate Algebra,Advanced Math Competitions; International Olym...,test/intermediate_algebra/1508.json,5,125,509,20,79


In [13]:
print(df.loc[:, [
    "problem_char_count",
    "solution_char_count",
    "subject_char_count",
    "level_char_count",
          ]].sum().sum())
df.loc[:, [
    "problem_char_count",
    "solution_char_count",
    "subject_char_count",
    "level_char_count",
          ]].sum()

398017


problem_char_count      97946
solution_char_count    265644
subject_char_count       6214
level_char_count        28213
dtype: int64

In [17]:
def translate_text(text, target_lang='UK'):
    if pd.isna(text):
        return text  # Return NaN values as is
    try:
        result = translator.translate_text(text, target_lang=target_lang)
        return result.text
    except deepl.DeepLException as e:
        print(f"Error translating text: {e}")
        return text  # Return the original text if translation fails

columns_to_translate = [
    'problem',
    'solution',
    'subject',
    'level',
]
target_language = 'UK'

for column in columns_to_translate:
    new_column_name = f"{column}_translated"
    df.loc[:, new_column_name] = df.loc[:, column].progress_apply(translate_text, target_lang=target_language)

# Save to CSV
output_file = '../datasets/math-500-uk.csv'
df.loc[:, ].to_csv(output_file, index=False)

df.loc[:, ]


100%|██████████| 500/500 [01:22<00:00,  6.04it/s]


,problem,solution,answer,subject,level,unique_id,original_level,problem_char_count,solution_char_count,subject_char_count,level_char_count,problem_translated,solution_translated,subject_translated,level_translated
0,"Convert the point $(0,3)$ in rectangular coord...",We have that $r = \sqrt{0^2 + 3^2} = 3.$ Also...,"\left( 3, \frac{\pi}{2} \right)",Precalculus,Foundational High-School Math,test/precalculus/807.json,2,161,439,11,29,"Переведіть точку $(0,3)$ у прямокутних координ...","Маємо, що $r = \sqrt{0^2 + 3^2} = 3.$ Також, я...",Попереднє обчислення,Фундаментальна математика для середньої школи
1,Define\n\[p = \sum_{k = 1}^\infty \frac{1}{k^2...,We count the number of times $\frac{1}{n^3}$ a...,p - q,Intermediate Algebra,Advanced Math Competitions; International Olym...,test/intermediate_algebra/1994.json,5,217,773,20,79,Визначити\n\[p = \sum_{k = 1}^\infty \frac{1}{...,"Підрахуємо, скільки разів $\frac{1}{n^3}$ зуст...",Проміжна алгебра,Поглиблені математичні змагання; міжнародна ол...
2,"If $f(x) = \frac{3x-2}{x-2}$, what is the valu...",$f(-2)+f(-1)+f(0)=\frac{3(-2)-2}{-2-2}+\frac{3...,\frac{14}{3},Algebra,High-School Intermediate Level Competition Mat...,test/algebra/2584.json,3,113,158,7,56,"Якщо $f(x) = \frac{3x-2}{x-2}$, то чому дорівн...",$f(-2)+f(-1)+f(0)=\frac{3(-2)-2}{-2-2}+\frac{3...,Алгебра,Олімпіада з математики для середнього рівня дл...
3,How many positive whole-number divisors does 1...,First prime factorize $196=2^2\cdot7^2$. The ...,9,Number Theory,High-School Intermediate Level Competition Mat...,test/number_theory/572.json,3,54,506,13,56,Скільки натуральних дільників має число 196?,Перший простий множник $196=2^2\cdot7^2$. Роз...,Теорія чисел,Олімпіада з математики для середнього рівня дл...
4,The results of a cross-country team's training...,Evelyn covered more distance in less time than...,\text{Evelyn},Algebra,Foundational High-School Math,test/algebra/1349.json,2,731,326,7,29,Нижче наведено результати тренувального забігу...,"Евелін пройшла більшу відстань за менший час, ...",Алгебра,Фундаментальна математика для середньої школи
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,What is the domain of the function $f(x) = \fr...,The inner logarithm is only defined if $x - 2 ...,"(2,12) \cup (12,102)",Intermediate Algebra,High-Level Math Competition Problems; Regional...,test/intermediate_algebra/1981.json,4,166,412,20,65,Яка область визначення функції $f(x) = \frac{2...,"Внутрішній логарифм визначено, якщо $x - 2 > 0...",Проміжна алгебра,Математичні олімпіадні задачі високого рівня; ...
496,Let $z = 1+i$ and $w = \dfrac{3z+1}{5z+7}$. Fi...,"Plugging in, we have $w = \dfrac{3(1+i)+1}{5(1...",\frac{5}{13},Intermediate Algebra,High-School Intermediate Level Competition Mat...,test/intermediate_algebra/1232.json,3,55,442,20,56,Нехай $z = 1+i$ і $w = \dfrac{3z+1}{5z+7}$. Зн...,"Підставляючи, маємо $w = \dfrac{3(1+i)+1}{5(1+...",Проміжна алгебра,Олімпіада з математики для середнього рівня дл...
497,An equiangular octagon has four sides of lengt...,The octagon can be partitioned into five squar...,\frac{7}{2},Geometry,Advanced Math Competitions; International Olym...,test/geometry/561.json,5,195,630,8,79,Рівносторонній восьмикутник має чотири сторони...,Восьмикутник можна розбити на п'ять квадратів ...,Геометрія,Поглиблені математичні змагання; міжнародна ол...
498,A sequence $(a_n)$ is defined as follows:\n\[a...,"First, if $a_3 = a_1,$ then\n\[a_1 = a_3 = a_5...",-1,Intermediate Algebra,Advanced Math Competitions; International Olym...,test/intermediate_algebra/1508.json,5,125,509,20,79,Послідовність $(a_n)$ визначається наступним ч...,"По-перше, якщо $a_3 = a_1,$ то\n\[a_1 = a_3 = ...",Проміжна алгебра,Поглиблені математичні змагання; міжнародна ол...


In [21]:
# Initialize the tokenizer
encoding = tiktoken.get_encoding("cl100k_base")

# Function to count tokens
def count_tokens(text):
    if pd.isna(text):
        return 0
    tokens = encoding.encode(text)
    return len(tokens)

# Apply the function to specified columns
columns_to_count_translated = [col + "_translated" for col in columns_to_count]
token_count_columns = []

for column in columns_to_count_translated:
    new_column_name = f"{column}_token_count"
    df[new_column_name] = df[column].apply(count_tokens)
    token_count_columns.append(new_column_name)

df.loc[:, token_count_columns].sum().sum()


np.int64(228035)

In [25]:
# price for text-embedding-3-small
print(228035 / 1_000_000 * 0.002)
# price for text-embedding-3-large
print(228035 / 1_000_000 * 0.13)
# price for ada v2
print(228035 / 1_000_000 * 0.1)

0.00045607
0.02964455
0.0228035


### Let's use ADA v2 model to generate our embeddings

In [52]:
openai_api_key = os.getenv("OPENAI_API_KEY")

In [48]:
df.loc[:, "embedding_text"] = df.loc[:, "level_translated"] + "\n" + df.loc[:, "subject_translated"] + "\n" + df.loc[:, "problem_translated"] + "\n" + df.loc[:, "solution_translated"]
df

,problem,solution,answer,subject,level,unique_id,original_level,problem_char_count,solution_char_count,subject_char_count,level_char_count,problem_translated,solution_translated,subject_translated,level_translated,problem_translated_token_count,solution_translated_token_count,subject_translated_token_count,level_translated_token_count,embedding_text
0,"Convert the point $(0,3)$ in rectangular coord...",We have that $r = \sqrt{0^2 + 3^2} = 3.$ Also...,"\left( 3, \frac{\pi}{2} \right)",Precalculus,Foundational High-School Math,test/precalculus/807.json,2,161,439,11,29,"Переведіть точку $(0,3)$ у прямокутних координ...","Маємо, що $r = \sqrt{0^2 + 3^2} = 3.$ Також, я...",Попереднє обчислення,Фундаментальна математика для середньої школи,80,238,12,23,Фундаментальна математика для середньої школи\...
1,Define\n\[p = \sum_{k = 1}^\infty \frac{1}{k^2...,We count the number of times $\frac{1}{n^3}$ a...,p - q,Intermediate Algebra,Advanced Math Competitions; International Olym...,test/intermediate_algebra/1994.json,5,217,773,20,79,Визначити\n\[p = \sum_{k = 1}^\infty \frac{1}{...,"Підрахуємо, скільки разів $\frac{1}{n^3}$ зуст...",Проміжна алгебра,Поглиблені математичні змагання; міжнародна ол...,122,434,10,56,Поглиблені математичні змагання; міжнародна ол...
2,"If $f(x) = \frac{3x-2}{x-2}$, what is the valu...",$f(-2)+f(-1)+f(0)=\frac{3(-2)-2}{-2-2}+\frac{3...,\frac{14}{3},Algebra,High-School Intermediate Level Competition Mat...,test/algebra/2584.json,3,113,158,7,56,"Якщо $f(x) = \frac{3x-2}{x-2}$, то чому дорівн...",$f(-2)+f(-1)+f(0)=\frac{3(-2)-2}{-2-2}+\frac{3...,Алгебра,Олімпіада з математики для середнього рівня дл...,77,97,5,34,Олімпіада з математики для середнього рівня дл...
3,How many positive whole-number divisors does 1...,First prime factorize $196=2^2\cdot7^2$. The ...,9,Number Theory,High-School Intermediate Level Competition Mat...,test/number_theory/572.json,3,54,506,13,56,Скільки натуральних дільників має число 196?,Перший простий множник $196=2^2\cdot7^2$. Роз...,Теорія чисел,Олімпіада з математики для середнього рівня дл...,25,299,7,34,Олімпіада з математики для середнього рівня дл...
4,The results of a cross-country team's training...,Evelyn covered more distance in less time than...,\text{Evelyn},Algebra,Foundational High-School Math,test/algebra/1349.json,2,731,326,7,29,Нижче наведено результати тренувального забігу...,"Евелін пройшла більшу відстань за менший час, ...",Алгебра,Фундаментальна математика для середньої школи,384,198,5,23,Фундаментальна математика для середньої школи\...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,What is the domain of the function $f(x) = \fr...,The inner logarithm is only defined if $x - 2 ...,"(2,12) \cup (12,102)",Intermediate Algebra,High-Level Math Competition Problems; Regional...,test/intermediate_algebra/1981.json,4,166,412,20,65,Яка область визначення функції $f(x) = \frac{2...,"Внутрішній логарифм визначено, якщо $x - 2 > 0...",Проміжна алгебра,Математичні олімпіадні задачі високого рівня; ...,85,257,10,50,Математичні олімпіадні задачі високого рівня; ...
496,Let $z = 1+i$ and $w = \dfrac{3z+1}{5z+7}$. Fi...,"Plugging in, we have $w = \dfrac{3(1+i)+1}{5(1...",\frac{5}{13},Intermediate Algebra,High-School Intermediate Level Competition Mat...,test/intermediate_algebra/1232.json,3,55,442,20,56,Нехай $z = 1+i$ і $w = \dfrac{3z+1}{5z+7}$. Зн...,"Підставляючи, маємо $w = \dfrac{3(1+i)+1}{5(1+...",Проміжна алгебра,Олімпіада з математики для середнього рівня дл...,38,228,10,34,Олімпіада з математики для середнього рівня дл...
497,An equiangular octagon has four sides of lengt...,The octagon can be partitioned into five squar...,\frac{7}{2},Geometry,Advanced Math Competitions; International Olym...,test/geometry/561.json,5,195,630,8,79,Рівносторонній восьмикутник має чотири сторони...,Восьмикутник можна розбити на п'ять квадратів ...,Геометрія,Поглиблені математичні змагання; міжнародна ол...,124,349,7,56,Поглиблені математичні змаганн

In [44]:
for i in range(10):

    print("Example #", i)
    print(df.loc[i, "embedding_text"])
    print()

Example # 0
Фундаментальна математика для середньої школи
Попереднє обчислення
Переведіть точку $(0,3)$ у прямокутних координатах у полярні координати.  Відповідь виведіть у вигляді $(r,\theta),$ де $r > 0$ і $0 \le \theta < 2 \pi.$
Маємо, що $r = \sqrt{0^2 + 3^2} = 3.$ Також, якщо провести пряму, яка з'єднує початок координат і $(0,3),$ то ця пряма утворює кут $\frac{\pi}{2}$ з додатною віссю $x$.

[asy]
unitsize(0.8 см);

draw((-0.5,0)--(3.5,0));
draw((0,-0.5)--(0,3.5));
draw(arc((0,0),3,0,90),red,Arrow(6));

dot((0,3), red);
label("$(0,3)$", (0,3), W);
dot((3,0), red);
[/asy]

Отже, полярні координати мають вигляд $\boxed{\left( 3, \frac{\pi}{2} \right)}.$

Example # 1
Поглиблені математичні змагання; міжнародна олімпіада або рівень університетських курсів.
Проміжна алгебра
Визначити
\[p = \sum_{k = 1}^\infty \frac{1}{k^2} \quad \text{and} \quad q = \sum_{k = 1}^\infty \frac{1}{k^3}.\]Знайдіть спосіб записати
\[\sum_{j = 1}^\infty \sum_{k = 1}^\infty \frac{1}{(j + k)^3}\]через $p$ т

In [54]:
client = OpenAI(api_key=openai_api_key)

def get_embedding(text, model="text-embedding-ada-002"):
    response = client.embeddings.create(input=text, model=model)
    return response.data[0].embedding

model = "text-embedding-ada-002"

df['embedding'] = df['embedding_text'].progress_apply(lambda x: get_embedding(x, model=model))
df

100%|██████████| 500/500 [03:08<00:00,  2.65it/s]


,problem,solution,answer,subject,level,unique_id,original_level,problem_char_count,solution_char_count,subject_char_count,...,problem_translated,solution_translated,subject_translated,level_translated,problem_translated_token_count,solution_translated_token_count,subject_translated_token_count,level_translated_token_count,embedding_text,embedding
0,"Convert the point $(0,3)$ in rectangular coord...",We have that $r = \sqrt{0^2 + 3^2} = 3.$ Also...,"\left( 3, \frac{\pi}{2} \right)",Precalculus,Foundational High-School Math,test/precalculus/807.json,2,161,439,11,...,"Переведіть точку $(0,3)$ у прямокутних координ...","Маємо, що $r = \sqrt{0^2 + 3^2} = 3.$ Також, я...",Попереднє обчислення,Фундаментальна математика для середньої школи,80,238,12,23,Фундаментальна математика для середньої школи\...,"[0.012237025424838066, 0.02391546033322811, -0..."
1,Define\n\[p = \sum_{k = 1}^\infty \frac{1}{k^2...,We count the number of times $\frac{1}{n^3}$ a...,p - q,Intermediate Algebra,Advanced Math Competitions; International Olym...,test/intermediate_algebra/1994.json,5,217,773,20,...,Визначити\n\[p = \sum_{k = 1}^\infty \frac{1}{...,"Підрахуємо, скільки разів $\frac{1}{n^3}$ зуст...",Проміжна алгебра,Поглиблені математичні змагання; міжнародна ол...,122,434,10,56,Поглиблені математичні змагання; міжнародна ол...,"[0.011512582190334797, -0.0008581626461818814,..."
2,"If $f(x) = \frac{3x-2}{x-2}$, what is the valu...",$f(-2)+f(-1)+f(0)=\frac{3(-2)-2}{-2-2}+\frac{3...,\frac{14}{3},Algebra,High-School Intermediate Level Competition Mat...,test/algebra/2584.json,3,113,158,7,...,"Якщо $f(x) = \frac{3x-2}{x-2}$, то чому дорівн...",$f(-2)+f(-1)+f(0)=\frac{3(-2)-2}{-2-2}+\frac{3...,Алгебра,Олімпіада з математики для середнього рівня дл...,77,97,5,34,Олімпіада з математики для середнього рівня дл...,"[0.016653820872306824, 0.014230269007384777, -..."
3,How many positive whole-number divisors does 1...,First prime factorize $196=2^2\cdot7^2$. The ...,9,Number Theory,High-School Intermediate Level Competition Mat...,test/number_theory/572.json,3,54,506,13,...,Скільки натуральних дільників має число 196?,Перший простий множник $196=2^2\cdot7^2$. Роз...,Теорія чисел,Олімпіада з математики для середнього рівня дл...,25,299,7,34,Олімпіада з математики для середнього рівня дл...,"[0.018865710124373436, 0.00984769593924284, -0..."
4,The results of a cross-country team's training...,Evelyn covered more distance in less time than...,\text{Evelyn},Algebra,Foundational High-School Math,test/algebra/1349.json,2,731,326,7,...,Нижче наведено результати тренувального забігу...,"Евелін пройшла більшу відстань за менший час, ...",Алгебра,Фундаментальна математика для середньої школи,384,198,5,23,Фундаментальна математика для середньої школи\...,"[-0.00832372810691595, 0.023255910724401474, -..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,What is the domain of the function $f(x) = \fr...,The inner logarithm is only defined if $x - 2 ...,"(2,12) \cup (12,102)",Intermediate Algebra,High-Level Math Competition Problems; Regional...,test/intermediate_algebra/1981.json,4,166,412,20,...,Яка область визначення функції $f(x) = \frac{2...,"Внутрішній логарифм визначено, якщо $x - 2 > 0...",Проміжна алгебра,Математичні олімпіадні задачі високого рівня; ...,85,257,10,50,Математичні олімпіадні задачі високого рівня; ...,"[0.0119700338691473, -0.0050599705427885056, -..."
496,Let $z = 1+i$ and $w = \dfrac{3z+1}{5z+7}$. Fi...,"Plugging in, we have $w = \dfrac{3(1+i)+1}{5(1...",\frac{5}{13},Intermediate Algebra,High-School Intermediate Level Competition Mat...,test/intermediate_algebra/1232.json,3,55,442,20,...,Нехай $z = 1+i$ і $w = \dfrac{3z+1}{5z+7}$. Зн...,"Підставляючи, маємо $w = \dfrac{3(1+i)+1}{5(1+...",Проміжна алгебра,Олімпіада з математики для середнього рівня дл...,38,228,10,34,Олімпіада з математики для середнього рівня дл...,"[0.003386223455891013, 0.018625861033797264, -..."
497,An equiangular octagon has four sides of

In [ ]:
df.to_csv("../datasets/math-500-uk-with-embeddings.csv", index=False)

In [90]:
inference_df = df[:490]
test_df = df[490:]

inference_df.to_csv("../datasets/math-500-uk-inference.csv", index=False)
test_df.to_csv("../datasets/math-500-uk-test.csv", index=False)

# Convert embeddings to a numpy array
embeddings = np.vstack(inference_df['embedding'].values).astype('float32')

# Normalize the embeddings to unit length
faiss.normalize_L2(embeddings)

# Initialize a FAISS index for inner product search
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)

# Add normalized embeddings to the index
index.add(embeddings)


In [91]:
# Example query
query = "Найпростіша задачка по геометрії"
query_embedding = np.array(get_embedding(query, model=model)).astype('float32').reshape(1, -1)

# Normalize the query embedding
faiss.normalize_L2(query_embedding)

# Define the number of nearest neighbors to retrieve
k = 25

# Perform the search to retrieve top k similar embeddings
distances, indices = index.search(query_embedding, k)

# Convert distances to cosine similarities
# Since embeddings are normalized, the inner product is equivalent to cosine similarity
cosine_similarities = distances[0]

# Define the similarity threshold (e.g., 0.8 for cosine similarity)
similarity_threshold = 0.7

# Filter results based on the similarity threshold
filtered_indices = indices[0][cosine_similarities >= similarity_threshold]
filtered_similarities = cosine_similarities[cosine_similarities >= similarity_threshold]

# Retrieve corresponding texts
similar_texts = inference_df.iloc[filtered_indices]
similar_texts

,problem,solution,answer,subject,level,unique_id,original_level,problem_char_count,solution_char_count,subject_char_count,...,problem_translated,solution_translated,subject_translated,level_translated,problem_translated_token_count,solution_translated_token_count,subject_translated_token_count,level_translated_token_count,embedding_text,embedding
308,Let $ABCD$ be a regular tetrahedron with side ...,The plane intersects each face of the tetrahed...,1+2\sqrt{3},Geometry,Advanced Math Competitions; International Olym...,test/geometry/711.json,5,203,375,8,...,Нехай $ABCD$ - правильний тетраедр зі стороною...,Площина перетинає кожну грань тетраедра по сер...,Геометрія,Поглиблені математичні змагання; міжнародна ол...,117,191,7,56,Поглиблені математичні змагання; міжнародна ол...,"[0.008417361415922642, 0.000720419455319643, 5..."
222,The area of $\triangle ABC$ is 6 square centim...,"Since $AB \parallel DE,$ we know that $\angle ...",54,Geometry,High-Level Math Competition Problems; Regional...,test/geometry/547.json,4,342,424,8,...,Площа $\трикутника ABC$ дорівнює 6 квадратних ...,"Оскільки $AB \паралельний DE,$ ми знаємо, що $...",Геометрія,Математичні олімпіадні задачі високого рівня; ...,176,272,7,50,Математичні олімпіадні задачі високого рівня; ...,"[0.010712559334933758, -0.007907348684966564, ..."
277,What is $\sqrt{53}$ in simplest radical form?,"Since 53 is prime, $\boxed{\sqrt{53}}$ is alre...",\sqrt{53},Prealgebra,High-Level Math Competition Problems; Regional...,test/prealgebra/1834.json,4,45,75,10,...,Що таке $\sqrt{53}$ у найпростішому радикально...,"Оскільки 53 - просте число, $\boxed{\sqrt{53}}...",Преалгебра,Математичні олімпіадні задачі високого рівня; ...,31,44,6,50,Математичні олімпіадні задачі високого рівня; ...,"[0.0058866022154688835, 0.016528010368347168, ..."
239,The parallelogram bounded by the lines $y=ax+c...,Two vertices of the first parallelogram are at...,16,Geometry,Advanced Math Competitions; International Olym...,test/geometry/229.json,5,296,1317,8,...,"Паралелограм, обмежений прямими $y=ax+c$, $y=a...",Дві вершини першого паралелограма знаходяться ...,Геометрія,Поглиблені математичні змагання; міжнародна ол...,155,705,7,56,Поглиблені математичні змагання; міжнародна ол...,"[-0.005379538983106613, -0.005849181208759546,..."
154,"A gecko is in a room that is 12 feet long, 10 ...",[asy]\nimport three;\ncurrentprojection=orthog...,2\sqrt{113},Geometry,Advanced Math Competitions; International Olym...,test/geometry/880.json,5,546,3356,8,...,"Гекон знаходиться у кімнаті довжиною 12 футів,...",[asy]\nімпортувати три;\ncurrentprojection=ort...,Геометрія,Поглиблені математичні змагання; міжнародна ол...,329,1887,7,56,Поглиблені математичні змагання; міжнародна ол...,"[-0.005317579489201307, 0.01098331343382597, -..."
274,Let $S$ be the union of the set of all points ...,$S$ looks like a nonagon with slightly rounded...,18+2\pi,Geometry,Advanced Math Competitions; International Olym...,test/geometry/686.json,5,235,1677,8,...,Нехай $S$ - це об'єднання множини всіх точок в...,$S$ виглядає як нонагон зі злегка заокругленим...,Геометрія,Поглиблені математичні змагання; міжнародна ол...,141,948,7,56,Поглиблені математичні змагання; міжнародна ол...,"[0.00911207776516676, 0.010200942866504192, 0...."
169,What is the number of square units in the area...,"This triangle is isosceles, and so the altitud...",12,Geometry,Foundational High-School Math,test/geometry/802.json,2,98,337,8,...,Скільки квадратних одиниць міститься у площі т...,"Цей трикутник рівнобедрений, тому висота, пров...",Геометрія,Фундаментальна математика для середньої школи,62,225,7,23,Фундаментальна математика для середньої школи\...,"[0.021239973604679108, 0.022146791219711304, 0..."
418,What is the shortest distance from the point $...,"The shortest line from the point $(6,0)$ to th...",2\sqrt{5},Algebra,Advanced Math Competitions; International Olym...,test/algebra/1796.json,5,123,933,7,...,"Яка найкоротша відстань від точки $(6, 0)$ до ...","

In [92]:
faiss.write_index(index, '../datasets/math-500-uk.index')